In [0]:
df=spark.read.csv(("/FileStore/tables/mohfw.csv"),header=True,inferSchema=True)
df.display()

S. No.,State/UT,1st Dose,2nd Dose,Total Doses
1,A & N Islands,"22,919","5,350","28,269"
2,Andhra Pradesh,"29,24,901","4,65,089","33,89,990"
3,Arunachal Pradesh,"88,250","25,781","1,14,031"
4,Assam,"11,38,046","2,06,862","13,44,908"
5,Bihar,"35,41,310","4,84,859","40,26,169"
6,Chandigarh,"86,126","15,613","1,01,739"
7,Chhattisgarh,"30,58,059","3,69,790","34,27,849"
8,Dadra & Nagar Haveli,"24,969","4,954","29,923"
9,Daman & Diu,"24,059","2,861","26,920"
10,Delhi,"14,81,156","3,18,250","17,99,406"


In [0]:
from pyspark.sql.functions import col

In [0]:
lis=[]
def func(res1):
  lis.append(res1)
  
for i in range(38):
  res = df.select("Total Doses").rdd.map(lambda x: x[0])
  res1=res.collect()[i]
  func(res1)
a=[x.replace(",","")for x in lis]
b=[int(x) for x in a]
print(b)

[28269, 3389990, 114031, 1344908, 4026169, 101739, 3427849, 29923, 26920, 1799406, 150177, 8147689, 2036392, 778402, 1095745, 2080318, 5230642, 4149413, 70021, 9719, 4973398, 8949560, 125612, 129891, 82170, 94224, 3617561, 87581, 1478280, 8287840, 95185, 3292109, 1685242, 767029, 7847622, 1114026, 7055520, 2478101]

In [0]:
df.createTempView("Vaccine")

In [0]:
def stripDQ(string):
  return string.replace(',', "") 
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType, IntegerType
udf_stripDQ = udf(stripDQ, StringType())
results1 = df.withColumn("1st Dose ", udf_stripDQ(df["1st Dose "]) ).withColumn("2nd Dose", udf_stripDQ(df["2nd Dose"]) ).withColumn("Total Doses", udf_stripDQ(df["Total Doses"]) )
results1.show()

+------+--------------------+---------+--------+-----------+
S. No.| State/UT|1st Dose |2nd Dose|Total Doses|
+------+--------------------+---------+--------+-----------+
 1| A & N Islands| 22919| 5350| 28269|
 2| Andhra Pradesh| 2924901| 465089| 3389990|
 3| Arunachal Pradesh| 88250| 25781| 114031|
 4| Assam| 1138046| 206862| 1344908|
 5| Bihar| 3541310| 484859| 4026169|
 6| Chandigarh| 86126| 15613| 101739|
 7| Chhattisgarh| 3058059| 369790| 3427849|
 8|Dadra & Nagar Haveli| 24969| 4954| 29923|
 9| Daman & Diu| 24059| 2861| 26920|
 10| Delhi| 1481156| 318250| 1799406|
 11| Goa| 124443| 25734| 150177|
 12| Gujarat| 7224754| 922935| 8147689|
 13| Haryana| 1863517| 172875| 2036392|
 14| Himachal Pradesh| 680428| 97974| 778402|
 15| Jammu & Kashmir| 920603| 175142| 1095745|
 16| Jharkhand| 1804732| 275586| 2080318|
 17| Karnataka| 4717382| 513260| 5230642|
 18| Kerala| 3706748| 442665| 4149413|
 19| Ladakh| 62881| 7140| 70021|
 20| Lakshadweep| 7566| 2153| 9719|
+------+--------------------+---------+--------+-----------+
only showing top 20 rows

In [0]:
results2 = results1.select( results1["State/UT"], results1["1st Dose "].cast(IntegerType()).alias("1st Dose"), results1["2nd Dose"].cast(IntegerType()).alias("2nd Dose"),results1["Total Doses"].cast(IntegerType()).alias("Total Doses") )
results2.sort(col("Total Doses").desc()).display()

State/UT,1st Dose,2nd Dose,Total Doses
Maharashtra,8075190,874370,8949560
Rajasthan,7303354,984486,8287840
Gujarat,7224754,922935,8147689
Uttar Pradesh,6672131,1175491,7847622
West Bengal,6252846,802674,7055520
Karnataka,4717382,513260,5230642
Madhya Pradesh,4402970,570428,4973398
Kerala,3706748,442665,4149413
Bihar,3541310,484859,4026169
Odisha,3178508,439053,3617561


In [0]:
results2.columns

Out[100]: ['State/UT', '1st Dose', '2nd Dose', 'Total Doses']

In [0]:
dell = results2.select('State/UT', '1st Dose', '2nd Dose', 'Total Doses',(col("2nd Dose")/col("1st Dose")*100).alias("Conversion Rate")).sort(col("Conversion Rate").desc())
dell.display()


State/UT,1st Dose,2nd Dose,Total Doses,Conversion Rate
Miscellaneous,1531779,946322,2478101,61.779277558969014
Manipur,86069,39543,125612,45.94337101627764
Meghalaya,94058,35833,129891,38.09670628760977
Nagaland,69379,24845,94224,35.81054786030355
Arunachal Pradesh,88250,25781,114031,29.21359773371105
Mizoram,63880,18290,82170,28.631809643080775
Lakshadweep,7566,2153,9719,28.45625165212794
A & N Islands,22919,5350,28269,23.343077795715345
Delhi,1481156,318250,1799406,21.486595605054433
Goa,124443,25734,150177,20.67934717099395
